In [ ]:
import re
from collections import defaultdict
# 4月9日
def convert_to_boolean_mode(keyword: str) -> str:
    """
    将用户输入的关键词转换为 MySQL Boolean Mode 格式。
    例如："NOT 王 AND 李 OR 张" → "-王 +李 张"
    """
    keyword = keyword.replace("(", "").replace(")", "").upper()
    # 将 NOT 替换为负号
    keyword = re.sub(r'\bNOT\b\s*', '-', keyword)
    # 保留 AND（用于前缀加号）
    keyword = re.sub(r'\bAND\b', 'AND', keyword)
    # 移除 OR（MySQL 中空格相当于 OR）
    keyword = re.sub(r'\bOR\b', '', keyword)
    tokens = keyword.split()

    converted = []
    prev_token = ""
    for token in tokens:
        if token == "AND":
            prev_token = "AND"
        elif token.startswith('-'):
            # 已经是 NOT 前缀的，直接添加
            converted.append(token)
            prev_token = ""
        else:
            # AND 后的词添加前缀 "+"，否则直接添加
            converted.append(f"+{token}" if prev_token == "AND" else token)
            prev_token = ""
    return " ".join(converted)


In [ ]:
# 4月9日
def convert_to_boolean_mode(keyword: str) -> str:
    """
    将用户输入的关键词转换为 MySQL Boolean Mode 格式。
    例如："NOT 王 AND 李 OR 张" → "-王 +李 张"
    """
    keyword = keyword.replace("(", "").replace(")", "").upper()
    keyword = re.sub(r'\bNOT\b\s*', '-', keyword)
    keyword = re.sub(r'\bAND\b', 'AND', keyword)
    keyword = re.sub(r'\bOR\b', '', keyword)
    tokens = keyword.split()

    converted = []
    prev_token = ""
    for token in tokens:
        if token == "AND":
            prev_token = "AND"
        elif token.startswith('-'):
            converted.append(token)
            prev_token = ""
        else:
            converted.append(f"+{token}" if prev_token == "AND" else token)
            prev_token = ""
    return " ".join(converted)

In [ ]:
# 4月9日
def build_where_clause_grouped(conditions):
    """
    根据 OR 逻辑将搜索条件分组，每个组内按列名再分组，并根据 AND/NOT 构造最终的 WHERE 子句。
    """

    def split_conditions_by_or(conditions):
        """根据 OR 逻辑将条件切分为多个逻辑块"""
        blocks = []
        current_block = []

        for i, cond in enumerate(conditions):
            logic = cond["logic"].strip().upper()
            if i == 0:
                current_block.append(cond)
            elif logic == "OR":
                blocks.append(current_block)
                current_block = [cond]
            else:
                current_block.append(cond)

        if current_block:
            blocks.append(current_block)

        return blocks

    def process_block(block):
        """处理单个逻辑块：按列分组后，根据 AND/NOT 构造 MATCH 子句"""
        grouped = defaultdict(list)

        for cond in block:
            grouped[cond["column"]].append((cond["logic"].upper().strip(), cond["keyword"]))

        sub_queries = []

        for col, cond_list in grouped.items():
            col_alias = {
                "doc_title": "d.doc_title",
                "author_name": "a.author_name",
                "title_name": "t.title_name",
                "full_text": "f.full_text"
            }.get(col)

            if not col_alias:
                continue

            clauses = []
            for i, (logic, keyword) in enumerate(cond_list):
                boolean_keyword = convert_to_boolean_mode(keyword)
                base = f"MATCH({col_alias}) AGAINST('{boolean_keyword}' IN BOOLEAN MODE)"

                if i == 0:
                    if logic == "NOT":
                        clauses.append(f"NOT {base}")
                    else:
                        clauses.append(base)
                else:
                    if logic == "AND":
                        clauses.append(f"AND {base}")
                    elif logic == "NOT":
                        clauses.append(f"AND NOT {base}")
                    # OR 已在外层处理，此处忽略

            if clauses:
                sub_queries.append(" ".join(clauses))

        return " AND ".join(sub_queries)

    # 1. 按 OR 逻辑拆分成多个逻辑块
    blocks = split_conditions_by_or(conditions)

    # 2. 对每个块生成对应的 WHERE 子句
    where_clauses = []
    for block in blocks:
        clause = process_block(block)
        if clause:
            where_clauses.append(f"({clause})")

    # 3. 将所有子句用 OR 连接成最终 WHERE 子句
    return " OR ".join(where_clauses)


In [ ]:
# 4月9日
def build_sql_with_joins(conditions):
    from_clause = "FROM documents d"
    joins = []
    select_fields = ["d.*"]

    # 获取所有使用到的列
    used_columns = {cond["column"] for cond in conditions}

    # 如果使用到作者信息，添加相关的连接
    if "author_name" in used_columns:
        joins.append("LEFT JOIN document_author_links dal ON dal.doc_id = d.doc_id")
        joins.append("LEFT JOIN authors a ON a.author_id = dal.author_id")
        select_fields.append("a.*, dal.*")

    # 如果使用到标题，添加 titles 表连接
    if "title_name" in used_columns:
        joins.append("LEFT JOIN titles t ON t.doc_id = d.doc_id")
        select_fields.append("t.*")

    # 如果使用到全文，添加 full_text_1 表连接
    if "full_text" in used_columns:
        joins.append("LEFT JOIN full_text_1 f ON f.doc_id = d.doc_id")
        select_fields.append("f.*")

    # 生成 WHERE 条件子句
    where_clause = build_where_clause_grouped(conditions)

    # 基于 AND 的逻辑分组分析
    def get_and_clusters(conditions):
        clusters = []
        current_cluster = []

        for i, cond in enumerate(conditions):
            if i == 0 or cond["logic"].upper().strip() == "AND":
                current_cluster.append(cond["column"])
            else:
                if current_cluster:
                    clusters.append(set(current_cluster))
                current_cluster = [cond["column"]]
        if current_cluster:
            clusters.append(set(current_cluster))
        return clusters

    clusters = get_and_clusters(conditions)

    # 如果在同一个 AND 组中同时使用 full_text 和 title_name，则需要用 title_id 关联
    needs_title_id_match = any({"full_text", "title_name"}.issubset(cluster) for cluster in clusters)

    if needs_title_id_match:
        extra_condition = "f.title_id = t.title_id"
        if where_clause:
            where_clause += f" AND {extra_condition}"
        else:
            where_clause = extra_condition

    # 组装最终 SQL 语句
    sql = f"SELECT DISTINCT {', '.join(select_fields)}\n{from_clause}"
    if joins:
        sql += "\n" + "\n".join(joins)
    if where_clause:
        sql += "\nWHERE " + where_clause
    sql += "\nLIMIT 100;"

    return sql


In [ ]:
# 4月9日
def fetch_extra_fields_by_doc_id(conn, doc_id, title_id=None):
    """
    基于 doc_id 获取代表性信息；如果提供了 title_id，则优先使用 title_id 获取 full_text 段落摘要。
    """
    enriched = {}

    with conn.cursor() as cursor:
        # 获取 title_name（最多两个标题，用于判断是否添加“等”）
        cursor.execute("SELECT title_name FROM titles WHERE doc_id = %s LIMIT 2", (doc_id,))
        rows = cursor.fetchall()
        if rows:
            title = rows[0]["title_name"]
            if len(rows) > 1:
                title += " 等"
            enriched["title_name"] = title
        else:
            enriched["title_name"] = None

        # 获取 author_name 和 author_org（最多两位作者，判断是否添加“等”）
        cursor.execute("""
            SELECT a.author_name, a.author_org
            FROM authors a
            JOIN document_author_links dal ON dal.author_id = a.author_id
            WHERE dal.doc_id = %s
            LIMIT 2
        """, (doc_id,))
        rows = cursor.fetchall()
        if rows:
            name = rows[0]["author_name"]
            if len(rows) > 1:
                name += " 等"
            enriched["author_name"] = name
            enriched["author_org"] = rows[0]["author_org"]
        else:
            enriched["author_name"] = None
            enriched["author_org"] = None

        # 获取 full_text（优先使用 title_id，否则使用 doc_id），截取前100字加“···”
        if title_id:
            cursor.execute("""
                SELECT full_text 
                FROM full_text_1 
                WHERE title_id = %s 
                ORDER BY full_text_order ASC 
                LIMIT 1
            """, (title_id,))
        else:
            cursor.execute("""
                SELECT full_text 
                FROM full_text_1 
                WHERE doc_id = %s 
                ORDER BY full_text_order ASC 
                LIMIT 1
            """, (doc_id,))

        row = cursor.fetchone()
        if row and row["full_text"]:
            enriched["full_text"] = row["full_text"][:100] + "···"
        else:
            enriched["full_text"] = None

    return enriched

In [ ]:
# 4月9日
def enrich_results_with_missing_info(conn, results):
    """
    如果查询结果中缺少必要字段，则根据 doc_id 补充这些字段的代表值。
    """
    must_fields = ["title_name", "author_name", "author_org", "full_text"]
    enriched_results = []

    for row in results:
        doc_id = row["doc_id"]

        # 检查哪些必要字段不存在于当前结果行中
        missing_fields = [field for field in must_fields if field not in row]

        if missing_fields:
            # 获取补充数据，只更新缺失的字段
            extra_data = fetch_extra_fields_by_doc_id(conn, doc_id)
            for key in missing_fields:
                row[key] = extra_data.get(key)

        enriched_results.append(row)

    return enriched_results


In [ ]:
import pymysql # 用的是不是Django？
import json

conn = pymysql.connect(
    host='localhost',
    user='root',
    password='前端同学请输入自己的数据库密码',
    db='leishu_yongle',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

cursor = conn.cursor()

# 这里就是从前端获取JSON查询数据的部分
# 请按照下面格式获取数据（以author_name为例）
# search_conditions = [
#     {"column": "author_name", "keyword": "姚廣孝", "logic": ""},
#     {"column": "full_text", "keyword": "大漠孤烟直", "logic": ""},
#     ]



sql = build_sql_with_joins(search_conditions)

try:
    cursor.execute(sql)
    results = cursor.fetchall()

    final_results = enrich_results_with_missing_info(conn, results)
    output_fields = ["doc_title","categoty_type","doc_specific_category","doc_style","doc_theme","author_name","author_org","role","title_name","full_text"]

    for row in results:
        filtered = {field: row.get(field, "") for field in output_fields}
        print(json.dumps(filtered, ensure_ascii=False, indent=2, default=str))

finally:
    cursor.close()
    conn.close()


{
  "doc_title": "《永乐大典》",
  "categoty_type": "",
  "doc_specific_category": "综合性类书",
  "doc_style": "类事",
  "doc_theme": "百科",
  "author_name": "解縉",
  "author_org": "明朝翰林院",
  "role": "总纂",
  "title_name": "永樂大典卷九百 等",
  "full_text": "敬齋古今黈···"
}
